In [1]:
import tensorflow as tf
import os
import cv2
import numpy as np
from keras import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, Input, Concatenate, GRU
import matplotlib.pyplot as plt
import tqdm
from sklearn.preprocessing import LabelBinarizer
import pathlib
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet121

2023-04-19 10:53:55.909237: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 10:53:56.879625: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/christian/miniconda3/envs/tf/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [2]:
MMI_emotions =  pathlib.Path("C:/Users/cdr03/Documents/Thesis/dataset/MMI/Emotions")
SEQUENCE_LENGTH = 5

In [3]:
def list_files_recursive(path, format = ".txt"):
    txt_files = set()  # create an empty set to store unique txt file names
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(format):
                file_path = os.path.join(root, file)
                txt_files.add(file_path)  # add the file path to the set
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            list_files_recursive(dir_path)  # recursively call the function on each subdirectory
    return list(txt_files)  # return a list of unique txt file names

#MMI_emotion_videos = list_files_recursive(MMI_emotions, ".avi")

In [4]:
np.shape(MMI_emotion_videos)

NameError: name 'MMI_emotion_videos' is not defined

In [75]:
def image_to_array(img, size):
    img_array = np.array(img , dtype="float32")/255
    img_array = img_array.reshape(size, size, 1)
    # Return the numpy array
    return img_array

In [64]:
size = 96
def detect_and_crop_face(img,resize=None):
    # Load the input image
    	
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Load the Haar Cascade classifier for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5)

    # Crop the image to contain the detected face (if any)
    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        face_image = image[y:y+h, x:x+w]
        face_image = cv2.resize(face_image, (size, size))
        face_image = image_to_array(face_image, size)
    else:
        rotated=cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        rotated_faces = face_cascade.detectMultiScale(rotated, scaleFactor=1.1, minNeighbors=5)
        if len(rotated_faces) > 0:
            (x, y, w, h) = rotated_faces[0]
            face_image = rotated[y:y+h, x:x+w]
            face_image = cv2.resize(face_image, (size, size))
            face_image = image_to_array(face_image, size)
        else:
            face_image = None
            print("face wasn't detected")
    return face_image

In [7]:
# https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-TensorFlow-2/blob/master/Chapter08/ch8_nb1_action_recognition.ipynb
video_paths = MMI_emotion_videos
print(type(MMI_emotion_videos))
IMG_size = (size, size)
def frame_generator():
    np.random.shuffle(video_paths)
    for video_path in video_paths:
        frames = []
        cap = cv2.VideoCapture(video_path)
        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        sample_every_frame = max(1, num_frames // SEQUENCE_LENGTH)
        current_frame = 0

        label = os.path.basename(video_path)

        max_images = SEQUENCE_LENGTH
        while True:
            success, frame = cap.read()
            if not success:
                break

            if current_frame % sample_every_frame == 0:
                # OPENCV reads in BGR, tensorflow expects RGB so we invert the order
                face_image_frame = detect_and_crop_face(frame, size)
                if face_image_frame is not None:
                    max_images -= 1
                    yield face_image_frame, video_path 
                else:
                    pass
            if max_images == 0:
                break
            current_frame += 1

# `from_generator` might throw a warning, expected to disappear in upcoming versions:
# https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/data/Dataset#for_example_2
dataset = tf.data.Dataset.from_generator(frame_generator,
             output_types=(tf.float32, tf.string),
             output_shapes=((size, size,1), ()))

dataset = dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)

<class 'list'>


CK+

In [70]:
size = 96
def listdirs(rootdir):
    list = []
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            list.append(d)
    return list

In [71]:
import glob
CK_sequences_folders = pathlib.Path("./cohn-kanade-images/")
IMG_size = (size, size)
parent_folder_paths = listdirs(CK_sequences_folders)
print(parent_folder_paths)
def frame_generator():
    np.random.shuffle(parent_folder_paths)
    for parent_folder_path in parent_folder_paths:
        subfolder_paths = sorted(glob.glob(os.path.join(parent_folder_path, '*')))
        for subfolder_path in subfolder_paths:
            image_sequence = sorted(glob.glob(os.path.join(subfolder_path, '*.png')))
            frames = []
            num_frames = len(image_sequence)
            if num_frames > SEQUENCE_LENGTH:
                image_sequence = image_sequence[(num_frames-SEQUENCE_LENGTH):]

            sample_every_frame = 1
            current_frame = 0
            max_images = SEQUENCE_LENGTH
            for image_path in image_sequence:
                # Load image using OpenCV
                frame = cv2.imread(image_path)
                if current_frame % sample_every_frame == 0:
                    # OPENCV reads in BGR, tensorflow expects RGB so we invert the order
                    face_image_frame = detect_and_crop_face(frame, IMG_size)
                    if face_image_frame is not None:
                        max_images -= 1
                        yield face_image_frame, image_path
                    else:
                        pass
                if max_images == 0:
                    break
                current_frame += 1

dataset = tf.data.Dataset.from_generator(frame_generator,
             output_types=(tf.float32, tf.string),
             output_shapes=((size, size, 1), ()))
dataset = dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)

['cohn-kanade-images/S129', 'cohn-kanade-images/S124', 'cohn-kanade-images/S068', 'cohn-kanade-images/S134', 'cohn-kanade-images/S127', 'cohn-kanade-images/S139', 'cohn-kanade-images/S115', 'cohn-kanade-images/S098', 'cohn-kanade-images/S506', 'cohn-kanade-images/S046', 'cohn-kanade-images/S108', 'cohn-kanade-images/S158', 'cohn-kanade-images/S093', 'cohn-kanade-images/S050', 'cohn-kanade-images/S130', 'cohn-kanade-images/S136', 'cohn-kanade-images/S118', 'cohn-kanade-images/S061', 'cohn-kanade-images/S070', 'cohn-kanade-images/S011', 'cohn-kanade-images/S087', 'cohn-kanade-images/S044', 'cohn-kanade-images/S010', 'cohn-kanade-images/S084', 'cohn-kanade-images/S026', 'cohn-kanade-images/S133', 'cohn-kanade-images/S117', 'cohn-kanade-images/S132', 'cohn-kanade-images/S073', 'cohn-kanade-images/S022', 'cohn-kanade-images/S151', 'cohn-kanade-images/S149', 'cohn-kanade-images/S094', 'cohn-kanade-images/S099', 'cohn-kanade-images/S014', 'cohn-kanade-images/S060', 'cohn-kanade-images/S504', 

In [65]:
def change_input_layer(pretrained_model, first_conv, h, w, c):
    pretrained_config = pretrained_model.get_config()

    # Change the input shape from the model
    pretrained_config["layers"][0]["config"]["batch_input_shape"] = (None, h, w, c)

    updated_model = Model.from_config(pretrained_config)


    def avg_weights(weights):
        average_weights = np.mean(weights, axis=-2).reshape(weights[:, :, -1:, :].shape)
        return(average_weights)

    pretrained_updated_config = updated_model.get_config()
    pretrained_updated_layer_names = [pretrained_updated_config['layers'][x]['name'] for x in range(len(pretrained_updated_config['layers']))]
    print(pretrained_updated_layer_names)
    first_conv_name = pretrained_updated_layer_names[first_conv]

    for layer in pretrained_model.layers:
        if layer.name in pretrained_updated_layer_names:
            if layer.get_weights() != []:  #All convolutional layers and layers with weights (no input layer or any pool layers)
                target_layer = updated_model.get_layer(layer.name)
            
                if layer.name in first_conv_name:    #For the first convolutionl layer
                    weights = layer.get_weights()[0]
                    biases  = layer.get_weights()[1]
                    
                    weights_single_channel = avg_weights(weights)
                                                                
                    target_layer.set_weights([weights_single_channel, biases])  #Now set weights for the first conv. layer
                    target_layer.trainable = False   #You can make this trainable if you want. 
            
                else:
                    target_layer.set_weights(layer.get_weights())   #Set weights to all other layers. 
                    target_layer.trainable = False  #You can make this trainable if you want.
    updated_model.summary()
    return updated_model


In [66]:
vgg16_model = VGG16(include_top=False, weights='imagenet')
vgg16_updated = change_input_layer(vgg16_model, 1, size, size, 1)

['input_2', 'block1_conv1', 'block1_conv2', 'block1_pool', 'block2_conv1', 'block2_conv2', 'block2_pool', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_pool', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_pool', 'block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_pool']
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 1)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 96, 96, 64)        640       
                                                                 
 block1_conv2 (Conv2D)       (None, 96, 96, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 48, 48, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (

In [67]:
final_vgg16 = Sequential()
final_vgg16.add(vgg16_updated)
final_vgg16.add(Flatten())


In [9]:
def change_input_layer_dense(pretrained_model, first_conv, h, w, c):
    pretrained_config = pretrained_model.get_config()

    # Change the input shape from the model
    pretrained_config["layers"][0]["config"]["batch_input_shape"] = (None, h, w, c)

    updated_model = Model.from_config(pretrained_config)


    def avg_weights(weights):
        average_weights = np.mean(weights, axis=-2).reshape(weights[:, :, -1:, :].shape)
        return(average_weights)

    pretrained_updated_config = updated_model.get_config()
    pretrained_updated_layer_names = [pretrained_updated_config['layers'][x]['name'] for x in range(len(pretrained_updated_config['layers']))]
    print(pretrained_updated_layer_names)
    first_conv_name = pretrained_updated_layer_names[first_conv]

    for layer in pretrained_model.layers:
        if layer.name in pretrained_updated_layer_names:
            if layer.get_weights() != []:  #All convolutional layers and layers with weights (no input layer or any pool layers)
                target_layer = updated_model.get_layer(layer.name)
            
                if layer.name in first_conv_name:    #For the first convolutionl layer
                    weights = layer.get_weights()[0]
                    
                    weights_single_channel = avg_weights(weights)
                                                                
                    target_layer.set_weights([weights_single_channel])  #Now set weights for the first conv. layer
                    target_layer.trainable = False   #You can make this trainable if you want. 
            
                else:
                    target_layer.set_weights(layer.get_weights())   #Set weights to all other layers. 
                    target_layer.trainable = False  #You can make this trainable if you want.
    updated_model.summary()
    return updated_model


In [10]:
densenet_model = DenseNet121(include_top=False, weights='imagenet')
densenet_updated = change_input_layer_dense(densenet_model, 2, size, size, 1)

['input_1', 'zero_padding2d', 'conv1/conv', 'conv1/bn', 'conv1/relu', 'zero_padding2d_1', 'pool1', 'conv2_block1_0_bn', 'conv2_block1_0_relu', 'conv2_block1_1_conv', 'conv2_block1_1_bn', 'conv2_block1_1_relu', 'conv2_block1_2_conv', 'conv2_block1_concat', 'conv2_block2_0_bn', 'conv2_block2_0_relu', 'conv2_block2_1_conv', 'conv2_block2_1_bn', 'conv2_block2_1_relu', 'conv2_block2_2_conv', 'conv2_block2_concat', 'conv2_block3_0_bn', 'conv2_block3_0_relu', 'conv2_block3_1_conv', 'conv2_block3_1_bn', 'conv2_block3_1_relu', 'conv2_block3_2_conv', 'conv2_block3_concat', 'conv2_block4_0_bn', 'conv2_block4_0_relu', 'conv2_block4_1_conv', 'conv2_block4_1_bn', 'conv2_block4_1_relu', 'conv2_block4_2_conv', 'conv2_block4_concat', 'conv2_block5_0_bn', 'conv2_block5_0_relu', 'conv2_block5_1_conv', 'conv2_block5_1_bn', 'conv2_block5_1_relu', 'conv2_block5_2_conv', 'conv2_block5_concat', 'conv2_block6_0_bn', 'conv2_block6_0_relu', 'conv2_block6_1_conv', 'conv2_block6_1_bn', 'conv2_block6_1_relu', 'conv

In [11]:
final_dense = Sequential()
final_dense.add(densenet_updated)
final_dense.add(Flatten())

In [72]:
import numpy as np
from skimage.feature import hog

def extract_hog_features(images):
    features = []
    for image in images:
        fd, hog_image = hog(image, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True,channel_axis=-1)
        features.append(fd)
    return np.array(features)

In [73]:
from skimage.transform import resize
def resize_images(images):
    resized_images = []
    for image in images:
        resized_images.append(resize(image, (size, size)))
    return np.asarray(resized_images).astype('float32')

In [14]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mping

In [131]:
current_path = None
all_features = []
files = []
folder_save = "./CK+Sequences_vgg16/"

for img, batch_paths in tqdm.tqdm(dataset):
    batch_features = final_vgg16.predict(img)
    batch_features = tf.reshape(batch_features, 
                             (batch_features.shape[0], -1))
    
    for features, path in zip(batch_features.numpy(), batch_paths.numpy()):
        if os.path.dirname(path) != current_path and current_path is not None:
            output_path = folder_save+session[:8]+".npy"
            np.save(output_path, all_features)
            all_features = []
            files.append(output_path)
        current_path = os.path.dirname(path)
        #plt.imshow(features)
        #plt.show()
        session = os.path.basename(path).decode('utf-8')
        all_features.append(features)
        



0it [00:00, ?it/s]

1/1 [==============================] - 1s 701ms/step


1it [00:01,  1.83s/it]

1/1 [==============================] - 1s 526ms/step


2it [00:02,  1.40s/it]

...

184it [04:15,  1.31s/it]

1/1 [==============================] - 0s 415ms/step


185it [04:16,  1.27s/it]

1/1 [==============================] - 0s 60ms/step


186it [04:16,  1.38s/it]


In [77]:
SEQUENCE_LENGTH

5

In [78]:
longest_feature = len(max(all_features, key=len))
print(longest_feature)

4608


## MMI

In [200]:
current_path = None
all_features = []
files = []

for img, batch_paths in tqdm.tqdm(dataset):
    batch_features = final_dense(img)
    batch_features = tf.reshape(batch_features, 
                              (batch_features.shape[0], -1))
    
    for features, path in zip(batch_features.numpy(), batch_paths.numpy()):
        if path != current_path and current_path is not None:
            output_path = current_path.decode().replace('.avi', '.npy')
            np.save(output_path, all_features)
            all_features = []
            files.append(current_path)
            
        current_path = path
        all_features.append(features)
        



68it [03:24,  2.74s/it]

face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected


86it [04:27,  3.90s/it]

face wasn't detected


87it [04:30,  3.54s/it]

...

122it [06:47,  4.19s/it]

face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected


133it [07:35,  3.74s/it]

face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected
face wasn't detected


146it [08:18,  3.41s/it]


In [201]:
longest_feature = len(max(all_features, key=len))
print(longest_feature)

9216


In [202]:

MMI_Emotions = pd.read_pickle("./MMI_emotion_video_labels")

In [203]:
files_df = pd.DataFrame(files, columns=["Path"])

In [204]:
files_df["Name"] = files_df["Path"].apply(lambda x: x[-12:-4].decode('utf-8'))

In [205]:
files_df

,Path,Name
0,b'C:\\Users\\cdr03\\Documents\\Thesis\\dataset...,S032-002
1,b'C:\\Users\\cdr03\\Documents\\Thesis\\dataset...,S036-002
2,b'C:\\Users\\cdr03\\Documents\\Thesis\\dataset...,S050-003
3,b'C:\\Users\\cdr03\\Documents\\Thesis\\dataset...,S050-006
4,b'C:\\Users\\cdr03\\Documents\\Thesis\\dataset...,S001-115
...,...,...
229,b'C:\\Users\\cdr03\\Documents\\Thesis\\dataset...,S039-003
230,b'C:\\Users\\cdr03\\Documents\\Thesis\\dataset...,S040-007
231,b'C:\\Users\\cdr03\\Documents\\Thesis\\dataset...,S053-018
232,b'C:\\Users\\cdr03\\Documents\\Thesis\\dataset...,S045-007


In [206]:
files_df["Path"] = files_df["Path"].apply(lambda x: x.decode('utf-8'))

In [207]:
MMI_Emotions_labels = pd.merge(MMI_Emotions, files_df, how="inner")

In [208]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(MMI_Emotions_labels['Path'], MMI_Emotions_labels['Emotion'], test_size=0.2, random_state=12, stratify=MMI_Emotions_labels['Emotion'])

In [209]:
X_train = X_train.to_list()
X_val = X_val.to_list()

In [210]:
X_train

['C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1804\\S031-005.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1928\\S045-012.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1809\\S032-001.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1855\\S036-004.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1950\\S047-001.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1826\\S033-007.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1764\\S021-004.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1879\\S039-006.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\2017\\S053-013.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\575\\S005-105.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\217\\S002-099.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1835\\S034-005.

In [211]:
y_train = np.array(np.array(y_train).reshape(-1, 1))
y_val = np.array(np.array(y_val).reshape(-1, 1))

In [212]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False)
y_train = onehot_encoder.fit_transform(y_train)
y_val = onehot_encoder.fit_transform(y_val)

In [286]:
from sklearn.preprocessing import OneHotEncoder
emotions = [1,2,3,4,5,6]
onehot_encoder = LabelBinarizer(sparse_output=False)
encoder = onehot_encoder.fit(emotions)


def make_generator(file_list, label_list):
    def generator():
        np.random.shuffle(file_list)
        for path, label in zip(file_list, label_list):
            full_path = path.replace('.avi', '.npy')
            features = np.load(full_path)

            padded_sequence = np.zeros((SEQUENCE_LENGTH, longest_feature))
            padded_sequence[0:len(features)] = np.array(features)
            yield padded_sequence, label
    return generator

In [287]:
y_val

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0.

In [298]:
train_dataset = tf.data.Dataset.from_generator(make_generator(X_train,y_train),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=((SEQUENCE_LENGTH, longest_feature), (len(emotions))))
train_dataset = train_dataset.batch(8).prefetch(tf.data.experimental.AUTOTUNE)


valid_dataset = tf.data.Dataset.from_generator(make_generator(X_val,y_val),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=((SEQUENCE_LENGTH, longest_feature), (len(emotions))))
valid_dataset = valid_dataset.batch(8).prefetch(tf.data.experimental.AUTOTUNE)

## CK+

In [79]:
import pandas as pd

In [80]:
CK_emotion = pd.read_pickle(f"./CK_Data_96_Emotion_LF")

In [81]:
CK_emotion["Name"] = CK_emotion['Subject'] +"_"+ CK_emotion['Number'] 

In [82]:
CK_emotion = CK_emotion[CK_emotion.Emotion !=2 ]
CK_emotion.Emotion = CK_emotion.Emotion.replace(7,2)

In [83]:
files = list_files_recursive("./CK+Sequences_vgg16/", ".npy")

In [84]:
files_df = pd.DataFrame(files, columns=["Path"])

In [85]:
#files_df.to_pickle("./CK+Sequences/CNN_Emotion_LSTM")
#files_df = pd.read_pickle("./CK+sequences/CNN_Emotion_LSTM")

In [86]:
files_df

,Path
0,./CK+Sequences_vgg16/S158_002.npy
1,./CK+Sequences_vgg16/S115_003.npy
2,./CK+Sequences_vgg16/S056_004.npy
3,./CK+Sequences_vgg16/S102_009.npy
4,./CK+Sequences_vgg16/S081_002.npy
...,...
587,./CK+Sequences_vgg16/S119_006.npy
588,./CK+Sequences_vgg16/S042_001.npy
589,./CK+Sequences_vgg16/S128_004.npy
590,./CK+Sequences_vgg16/S052_004.npy


In [87]:
files_df['Name'] = files_df["Path"].apply(lambda x: x[-12:-4])

In [88]:
CK_Emotions_labels = pd.merge(CK_emotion, files_df, how="inner")

In [89]:
CK_Emotions_labels

,Subject,Number,Code,Image,Emotion,Name,Path
0,S032,005,00000016,"[[0.5294118, 0.5254902, 0.49803922, 0.39215687...",3,S032_005,./CK+Sequences_vgg16/S032_005.npy
1,S108,006,00000020,"[[0.13725491, 0.16470589, 0.19215687, 0.133333...",3,S108_006,./CK+Sequences_vgg16/S108_006.npy
2,S052,004,00000033,"[[0.38431373, 0.39215687, 0.39215687, 0.384313...",5,S052_004,./CK+Sequences_vgg16/S052_004.npy
3,S087,007,00000016,"[[0.21176471, 0.2, 0.20784314, 0.20784314, 0.1...",1,S087_007,./CK+Sequences_vgg16/S087_007.npy
4,S098,004,00000015,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",5,S098_004,./CK+Sequences_vgg16/S098_004.npy
...,...,...,...,...,...,...,...
303,S014,005,00000017,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7764706...",5,S014_005,./CK+Sequences_vgg16/S014_005.npy
304,S107,005,00000011,"[[0.2509804, 0.26666668, 0.22745098, 0.0705882...",3,S107_005,./CK+Sequences_vgg16/S107_005.npy
305,S060,003,00000018,"[[0.31764707, 0.30588236, 0.2901961, 0.2823529...",2,S060_003,./CK+Sequences_vgg16/S060_003.npy
306,S111,001,00000014,"[[0.1764706, 0.18039216, 0.16862746, 0.1647058...",2,S111_001,./CK+Sequences_vgg16/S111_001.npy


In [90]:
from sklearn.model_selection import train_test_split

def create_stratified_datasets(X, y, test_size=0.1, val_size=0.1, random_state=None):
    """
    Creates three stratified datasets - train, validation, and test - from the input data X and target y.
    
    Parameters:
        X (array-like): The input data.
        y (array-like): The target variable.
        test_size (float): The proportion of the data to include in the test set.
        val_size (float): The proportion of the remaining data to include in the validation set.
        random_state (int): Seed for the random number generator.
    
    Returns:
        A tuple containing the train, validation, and test datasets, each as a tuple of input and target variables.
    """
    
    # split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    
    # split remaining data into validation and train sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size/(1-test_size), random_state=random_state, stratify=y_train)
    
    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

In [91]:
from sklearn.model_selection import train_test_split
(X_train, y_train), (X_val, y_val), (X_test, y_test) =  create_stratified_datasets(CK_Emotions_labels['Path'], CK_Emotions_labels['Emotion'])

In [92]:
X_train = X_train.to_list()
X_test = X_test.to_list()
X_val = X_val.to_list()

In [93]:
len(X_train[1])

33

In [19]:
#longest_feature = 9216 # Densenet
#longest_feature = 4608 # VGG16

In [132]:
from sklearn.preprocessing import OneHotEncoder
emotions = [1,2,3,4,5,6]
onehot_encoder = OneHotEncoder(sparse_output=False)

folder_path = ".\CK+sequences_vgg16"


def make_generator(file_list, label_list):
    def generator():
        np.random.shuffle(file_list)
        for path, label in zip(file_list,label_list):
            features = np.load(path)
            
            padded_sequence = np.zeros((SEQUENCE_LENGTH, longest_feature))
            padded_sequence[0:len(features)] = np.array(features)

            #transformed_label = encoder.transform(label)
            yield padded_sequence, label
    return generator

In [21]:
test = np.load(CK_Emotions_labels.Path[1])


In [23]:
import numpy as np

In [95]:
y_train = np.array(np.array(y_train).reshape(-1, 1))
y_val = np.array(np.array(y_test).reshape(-1, 1))

In [96]:
onehot_encoder = OneHotEncoder(sparse_output=False)
y_train = onehot_encoder.fit_transform(y_train)
y_val = onehot_encoder.fit_transform(y_val)


In [26]:
SEQUENCE_LENGTH=5
longest_feature = 9216

In [133]:
train_dataset = tf.data.Dataset.from_generator(make_generator(X_train,y_train),
                 output_types=(tf.float32, tf.int32),
                 output_shapes=((SEQUENCE_LENGTH, longest_feature), (len(emotions))))
train_dataset = train_dataset.batch(16)


valid_dataset = tf.data.Dataset.from_generator(make_generator(X_val,y_val),
                 output_types=(tf.float32, tf.int32),
                 output_shapes=((SEQUENCE_LENGTH, longest_feature), (len(emotions))))
valid_dataset = valid_dataset.batch(16)

In [134]:
test_dataset = tf.data.Dataset.from_generator(make_generator(X_test,y_test),
                 output_types=(tf.float32, tf.int32),
                 output_shapes=((SEQUENCE_LENGTH, longest_feature), (len(emotions))))
test_dataset = valid_dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)

In [143]:
test = train_dataset

In [144]:
for x in test:
    print(x)

(<tf.Tensor: shape=(16, 5, 4608), dtype=float32, numpy=
array([[[7.3728126e-01, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 7.1868593e-01, 0.0000000e+00],
        [6.6871512e-01, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 7.3644269e-01, 0.0000000e+00],
        [6.1044413e-01, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 7.3400855e-01, 0.0000000e+00],
        [6.2345934e-01, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 7.1622705e-01, 0.0000000e+00],
        [6.3427162e-01, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 7.4143636e-01, 0.0000000e+00]],

       [[4.3391702e-01, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 7.1117979e-01, 0.0000000e+00],
        [4.6452367e-01, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 6.8344730e-01, 0.0000000e+00],
        [4.6056998e-01, 0.0000000e+00, 6.6965818e-04, ...,
         0.0000000e+00, 6.8457574e-01, 0.0000000e+00],
        [4.5971212e-01, 0.0000

## TRaining

In [99]:
precision = tf.keras.metrics.Precision(name="Precision")
recall = tf.keras.metrics.Recall(name="Recall")

In [100]:
from keras import backend as K

def custom_f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall


    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))

auc_metric = tf.keras.metrics.AUC(
num_thresholds=200, curve="ROC",
summation_method="interpolation",
thresholds=None, multi_label=False
)


In [101]:
def gru_ccc_loss():
        model = Sequential()
        model.add(GRU(100, return_sequences=True, input_shape=[SEQUENCE_LENGTH, longest_feature]))
        model.add(GRU(100, return_sequences=True))
        model.add(GRU(50, return_sequences=False))
        model.add(Dense(6))
        # lr_schedule = ExponentialDecay(
        #     initial_learning_rate=1e-2,
        #     decay_steps=400,
        #     decay_rate=0.97)
        # opt = Adam(learning_rate=lr_schedule)
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        return model

In [102]:
def keras_rnn():
    model = Sequential()
    model.add(tf.keras.layers.Masking(mask_value=0., input_shape=(SEQUENCE_LENGTH, longest_feature)))
    model.add(tf.keras.layers.GRU(512, dropout=0.5))
    model.add(Dense(6, activation="softmax"))

    model.compile(
        loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy", precision, recall, custom_f1, auc_metric]
    )
    return model

In [130]:
cnnrnn = keras_rnn()
cnnrnn.fit(train_dataset, callbacks = callbacks, epochs=20, validation_data=valid_dataset)


Epoch 1/20


2023-04-19 12:49:14.101283: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


     16/Unknown - 6s 144ms/step - loss: 2.6720 - accuracy: 0.2602 - Precision: 0.2484 - Recall: 0.1043 - custom_f1: 0.1642 - auc_1: 0.5734

2023-04-19 12:49:20.264193: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-19 12:49:22.362498: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-19 12:49:22.364204: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

INFO:tensorflow:Assets written to: ./chkpts/LSTM_Emotion_CK/assets


INFO:tensorflow:Assets written to: ./chkpts/LSTM_Emotion_CK/assets


16/16 [==============================] - 13s 574ms/step - loss: 2.6720 - accuracy: 0.2602 - Precision: 0.2484 - Recall: 0.1043 - custom_f1: 0.1642 - auc_1: 0.5734 - val_loss: 2.0416 - val_accuracy: 0.1613 - val_Precision: 1.0000 - val_Recall: 0.0323 - val_custom_f1: 0.0588 - val_auc_1: 0.5643 - lr: 0.0010
Epoch 2/20
16/16 [==============================] - 2s 152ms/step - loss: 1.9959 - accuracy: 0.2236 - Precision: 0.1579 - Recall: 0.0366 - custom_f1: 0.0476 - auc_1: 0.5904 - val_loss: 2.0175 - val_accuracy: 0.2581 - val_Precision: 0.1429 - val_Recall: 0.0323 - val_custom_f1: 0.0476 - val_auc_1: 0.5883 - lr: 0.0010
Epoch 3/20
16/16 [==============================] - 2s 151ms/step - loss: 1.8561 - accuracy: 0.2073 - Precision: 0.3125 - Recall: 0.0203 - custom_f1: 0.0397 - auc_1: 0.5768 - val_loss: 1.8018 - val_accuracy: 0.2581 - val_Precision: 0.1429 - val_Recall: 0.0323 - val_custom_f1: 0.0526 - val_auc_1: 0.6279 - lr: 0.0010
Epoch 4/20
16/16 [==============================] - 2s 151m

KeyboardInterrupt: 

In [149]:
from keras.layers import Dropout,Activation,BatchNormalization, Bidirectional, LSTM
def LSTM_Biderectional():
    regularizer = tf.keras.regularizers.l1_l2(0, 0.0001)
    model = Sequential(name="Convolutional_Biderectional_LSTM")

    # model.add(tf.keras.layers.Masking(mask_value=0., input_shape=[SEQUENCE_LENGTH, longest_feature]))
    model.add(Bidirectional(LSTM(128, return_sequences=True, dropout = 0.5)))
    model.add(Bidirectional(LSTM(64, return_sequences=False, dropout = 0.4)))
    model.add(Dense(512, kernel_regularizer=regularizer))
    model.add(Activation('elu'))
    model.add(Dropout(rate=0.7))
    model.add(Dense(units=6, activation="softmax",kernel_regularizer=regularizer, kernel_initializer='he_normal')) 
    model.compile(optimizer='Nadam',
                loss='categorical_crossentropy', metrics=[precision,recall,custom_f1, auc_metric,'accuracy'])

    return model
    
        


In [150]:
from keras.callbacks import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(
    monitor = 'val_custom_f1',
    factor=0.8,
    patience=7,
    min_lr = 1e-7,
    verbose=1

)

In [151]:
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='./chkpts/LSTM_Emotion_CK',monitor='val_custom_f1', mode='max',save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',patience=2)
#The callbacks list now becomes:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./clean_logs/Emotion/LSTM_Emotion_CK")
callbacks=[model_checkpoint_cb, lr_scheduler, tensorboard_callback]

In [152]:
cnnrnn = LSTM_Biderectional()
cnnrnn.fit(train_dataset, callbacks = callbacks, epochs=100, validation_data=valid_dataset)

Epoch 1/100


2023-04-19 13:19:00.265645: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-19 13:19:00.267505: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-19 13:19:00.268783: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

     16/Unknown - 10s 121ms/step - loss: 1.8118 - Precision: 0.5455 - Recall: 0.0128 - custom_f1: 0.0375 - auc_1: 0.6054 - accuracy: 0.2236       

2023-04-19 13:19:10.667639: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-19 13:19:10.669186: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-19 13:19:10.670435: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: ./chkpts/LSTM_Emotion_CK/assets


INFO:tensorflow:Assets written to: ./chkpts/LSTM_Emotion_CK/assets


16/16 [==============================] - 32s 2s/step - loss: 1.8118 - Precision: 0.5455 - Recall: 0.0128 - custom_f1: 0.0375 - auc_1: 0.6054 - accuracy: 0.2236 - val_loss: 1.7347 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_custom_f1: 0.0000e+00 - val_auc_1: 0.6495 - val_accuracy: 0.2581 - lr: 0.0010
Epoch 2/100
16/16 [==============================] - 2s 120ms/step - loss: 1.8080 - Precision: 0.3750 - Recall: 0.0122 - custom_f1: 0.0199 - auc_1: 0.6023 - accuracy: 0.2195 - val_loss: 1.7951 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_custom_f1: 0.0000e+00 - val_auc_1: 0.5928 - val_accuracy: 0.2581 - lr: 0.0010
Epoch 3/100
16/16 [==============================] - 2s 121ms/step - loss: 1.8308 - Precision: 0.0000e+00 - Recall: 0.0000e+00 - custom_f1: 0.0000e+00 - auc_1: 0.5808 - accuracy: 0.2033 - val_loss: 1.7429 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_custom_f1: 0.0000e+00 - val_auc_1: 0.6367 - val_accuracy: 0.2581 - lr: 0.0010
Epoch 4/100


KeyboardInterrupt: 

In [160]:

from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM
def gru_ccc_loss():
        model = Sequential()
        model.add(Bidirectional(LSTM(256, return_sequences=True, input_shape=[SEQUENCE_LENGTH, longest_feature], dropout=0.5)))
        model.add(Bidirectional(LSTM(128, return_sequences=False, dropout=0.5, recurrent_dropout=0.5)))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(6, activation='softmax'))
        lr_schedule = ExponentialDecay(
             initial_learning_rate=1e-2,
             decay_steps=400,
             decay_rate=0.97)
        opt = Adam(learning_rate=lr_schedule)
        model.compile(loss='categorical_crossentropy',
                      optimizer=opt,
                      metrics=['accuracy', precision, recall])

        return model


In [158]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

CudaSupportError: Error at driver init: 
Call to cuInit results in CUDA_ERROR_UNKNOWN (999):

In [161]:

model = gru_ccc_loss()
model.fit(train_dataset, epochs=100, validation_data=valid_dataset)

Epoch 1/100


2023-04-19 13:26:10.975591: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-19 13:26:10.978422: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-19 13:26:10.980579: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

     16/Unknown - 12s 195ms/step - loss: 1.9192 - accuracy: 0.2195 - Precision: 0.1351 - Recall: 0.0128

2023-04-19 13:26:22.995776: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-19 13:26:22.997598: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-19 13:26:22.998771: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

16/16 [==============================] - 13s 272ms/step - loss: 1.9192 - accuracy: 0.2195 - Precision: 0.1351 - Recall: 0.0128 - val_loss: 1.7411 - val_accuracy: 0.2581 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00
Epoch 2/100
16/16 [==============================] - 3s 203ms/step - loss: 1.7979 - accuracy: 0.1951 - Precision: 0.5000 - Recall: 0.0041 - val_loss: 1.7184 - val_accuracy: 0.2581 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00
Epoch 3/100
16/16 [==============================] - 3s 210ms/step - loss: 1.7449 - accuracy: 0.2602 - Precision: 0.0000e+00 - Recall: 0.0000e+00 - val_loss: 1.7096 - val_accuracy: 0.2581 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00
Epoch 4/100
16/16 [==============================] - 3s 204ms/step - loss: 1.7244 - accuracy: 0.2154 - Precision: 0.0000e+00 - Recall: 0.0000e+00 - val_loss: 1.7085 - val_accuracy: 0.2581 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00
Epoch 5/100
16/16 [==============================] - 3s 204ms/s

KeyboardInterrupt: 

In [65]:
model = tf.keras.Sequential([
tf.keras.layers.Masking(mask_value=0.),
tf.keras.layers.LSTM(1024, dropout=0.5, recurrent_dropout=0.5, return_sequences=True),
tf.keras.layers.LSTM(512, return_sequences=False),
Dense(6, activation='softmax')
])

In [66]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'top_k_categorical_accuracy', precision, recall])



In [67]:
model.fit(train_dataset, epochs=100, validation_data=valid_dataset)

Epoch 1/100


2023-04-17 12:32:56.086445: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 36.00MiB (rounded to 37748736)requested by op sequential_12/lstm_10/while/body/_1/sequential_12/lstm_10/while/lstm_cell_10/split
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-04-17 12:32:56.086495: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-04-17 12:32:56.086509: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 220, Chunks in use: 219. 55.0KiB allocated for chunks. 54.8KiB in use in bin. 1.6KiB client-requested in use in bin.
2023-04-17 12:32:56.086546: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in 

ResourceExhaustedError: Graph execution error:

Detected at node 'sequential_12/lstm_10/while/lstm_cell_10/split' defined at (most recent call last):
    File "/home/christian/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/christian/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/christian/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/christian/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/christian/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/christian/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
     

In [47]:
prediction_test = cnnrnn.predict(test_dataset)

2023-04-17 19:56:48.845634: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-17 19:56:48.846031: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


ValueError: in user code:

    File "/home/christian/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/home/christian/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/christian/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/home/christian/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/home/christian/miniconda3/envs/tf/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/christian/miniconda3/envs/tf/lib/python3.9/site-pack

In [42]:
prediction_test[1]
y_pred = np.argmax(prediction_test, axis=1)

In [43]:
y_pred

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2])

In [451]:
class_labels = np.argmax(y_val, axis=1)

In [452]:
class_labels

array([2, 3, 4, 5, 1, 4, 1, 4, 0, 0, 3, 5, 0, 1, 1, 2, 4, 2, 4, 4, 1, 4,
       0, 2, 0, 2, 1, 2, 1, 5, 2, 2, 2, 1, 1, 0, 1, 4, 3, 2, 0, 4, 1, 1,
       1, 2, 4, 4, 4, 0, 3, 1, 5, 4, 1, 3, 5, 4, 2, 1, 0, 5], dtype=int64)

In [54]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_val,prediction_test)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [110]:
SEQUENCE_LENGTH

10

In [102]:
def get_sequence_model():

    frame_features_input = tf.keras.Input((SEQUENCE_LENGTH, longest_feature))
    mask_input = tf.keras.Input((SEQUENCE_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://tf.keras.io/api/layers/recurrent_layers/gru/
    x = tf.keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = tf.keras.layers.GRU(8)(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(8, activation="relu")(x)
    output = tf.keras.layers.Dense(6, activation="softmax")(x)

    rnn_model = tf.keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

In [112]:
EPOCHS = 10
seq_model = get_sequence_model()
history = seq_model.fit(train_dataset,
                        epochs=EPOCHS, validation_data= valid_dataset)

Epoch 1/10


ValueError: in user code:

    File "c:\Users\cdr03\anaconda3\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\cdr03\anaconda3\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\cdr03\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\cdr03\anaconda3\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\cdr03\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\cdr03\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 219, in assert_input_compatibility
        raise ValueError(

    ValueError: Layer "m

In [109]:
seq_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 10, 9216)]   0           []                               
                                                                                                  
 input_9 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 gru_19 (GRU)                   (None, 10, 16)       443232      ['input_8[0][0]',                
                                                                  'input_9[0][0]']                
                                                                                                  
 gru_20 (GRU)                   (None, 8)            624         ['gru_19[0][0]']           

## Tuning

In [257]:
import keras_tuner

In [445]:
def build_model(hp):
    lrate = hp.Float('lrate', 1e-4, 1e-1, sampling='log')
    l1 = 0
    l2 = hp.Choice('l2',  values=[0.0, 1e-1, 1e-2, 1e-3, 1e-4])
    num_rnn = hp.Int('num_rnn', 32,512,32)
    num_hidden = hp.Int('num_hidden', 32,512,32)
    regularizer = tf.keras.regularizers.l1_l2(l1, l2)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Masking(mask_value=0.))
    model.add(tf.keras.layers.GRU(512, dropout=0.5,return_sequences=True, recurrent_dropout=0.5, input_shape= [SEQUENCE_LENGTH, longest_feature]))
    model.add(tf.keras.layers.GRU(256, dropout=0.5, recurrent_dropout=0.5))
    #model.add(tf.keras.layers.Dense(num_hidden, activation='relu', kernel_regularizer=regularizer))
    #model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(6, activation='softmax', kernel_regularizer=regularizer))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                  loss='categorical_crossentropy', metrics=[precision,recall,'accuracy', 'top_k_categorical_accuracy'])
    return model


In [446]:
tuner = keras_tuner.BayesianOptimization(
    build_model,
    objective=keras_tuner.Objective('val_accuracy', 'max'),
    max_trials = 10,
    overwrite=True
)

In [447]:
tuner.search(
    train_dataset, validation_data= valid_dataset,
    epochs=10
)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
0.093061          |0.093061          |lrate
0.01              |0.01              |l2
64                |64                |num_rnn
160               |160               |num_hidden

Epoch 1/10
123/123 [==============================] - 80s 597ms/step - loss: 8.8756 - Precision: 0.2402 - Recall: 0.1536 - accuracy: 0.2317 - top_k_categorical_accuracy: 0.8659 - val_loss: 5.7034 - val_Precision: 0.1803 - val_Recall: 0.1774 - val_accuracy: 0.1774 - val_top_k_categorical_accuracy: 0.9194
Epoch 2/10
123/123 [==============================] - 86s 696ms/step - loss: 6.3693 - Precision: 0.1916 - Recall: 0.1667 - accuracy: 0.1789 - top_k_categorical_accuracy: 0.8374 - val_loss: 4.7148 - val_Precision: 0.2581 - val_Recall: 0.2581 - val_accuracy: 0.2581 - val_top_k_categorical_accuracy: 0.8548
Epoch 3/10
123/123 [==============================] - 88s 718ms/step - loss: 7.1011 - Precision: 0.1739 - Recall: 0.1626 - accura

KeyboardInterrupt: 

In [279]:
len(y_train)

168

In [280]:
len(y_val)

43